# Convirtiendo datos a ttl
:D


In [44]:
import pandas as pd # no quitar, se rompe el proyectoimport pandas as pd

In [45]:
import requests

ISDEBUG = True

def obtener_codigo_wikidata(url):

    if ISDEBUG:
        return None

    try:
        # Extraer el título del artículo de la URL
        if not url.startswith("http://" or "https://"):
            raise Exception("URL inválida, debe comenzar con 'https://'")

        title = url.split("/wiki/")[-1]
        if not title:
            raise Exception("No se encontró el título del artículo en la URL proporcionada.")

        # Llamar a la API de MediaWiki para obtener datos de la entidad
        endpoint = "https://en.wikipedia.org/w/api.php"
        params = {
            "action": "query",
            "titles": title,
            "prop": "pageprops",
            "format": "json",
        }
        response = requests.get(endpoint, params=params)
        response.raise_for_status()

        data = response.json()
        # Extraer el código de Wikidata
        pages = data.get("query", {}).get("pages", {})
        for page_id, page_info in pages.items():
            if "pageprops" in page_info and "wikibase_item" in page_info["pageprops"]:
                return page_info["pageprops"]["wikibase_item"]

        raise Exception("No se encontró el código de Wikidata para la URL proporcionada.")

    except requests.exceptions.RequestException as e:
        raise Exception(f"Error al obtener los datos de Wikidata: {e}")

    except Exception as e:
        print(e)
        return None

# Ejemplo de uso
url = "https://en.wikipedia.org/wiki/Max_Verstappen"
codigo_wikidata = obtener_codigo_wikidata(url)
print(f"Código de Wikidata: {codigo_wikidata}")
filter_years = [2023]

Código de Wikidata: None


In [46]:
#Leemos TODO PORQUE SI
circuits = pd.read_csv('data/circuits.csv')
constructorResults = pd.read_csv('data/constructor_results.csv')
constructors = pd.read_csv('data/constructors.csv')
constructorStandings = pd.read_csv('data/constructor_standings.csv')
drivers = pd.read_csv('data/drivers.csv')
driverStandings = pd.read_csv('data/driver_standings.csv')
lapTimes = pd.read_csv('data/lap_times.csv')
pitStops = pd.read_csv('data/pit_stops.csv')
qualifying = pd.read_csv('data/qualifying.csv')
races = pd.read_csv('data/races.csv')
races_results = pd.read_csv('data/results.csv')
seasons = pd.read_csv('data/seasons.csv')
status = pd.read_csv('data/status.csv')
NAME_FILE = "data.ttl"

In [47]:
print("nueva, notacion usando las siguientes data classes, esta para \n1 mantener la ontologia limpia es decir no tener duplicas de definicion de algo \n2 hacer un solo flush en el archivo ttl los datos, lo que implicara una mejora en el tiempo de la generacion del archivo")

from dataclasses import dataclass, field
from typing import List, Union, Dict

@dataclass
class RDFPredicate:
    property: str
    objects: List[str]

@dataclass
class RDFStatement:
    _instances = {}
    subject: str
    predicates: dict[str, list[str]] = field(default_factory=dict)

    def __new__(cls, *args, **kwargs):
        # subject expected to either be the first args or with the key "subject"
        subject = kwargs.get("subject", args[0] if args else None)

        if not subject:
            raise ValueError("RDFStatement requires a 'subject' argument.")
    
        if subject not in cls._instances:
            new_instance = super(RDFStatement, cls).__new__(cls)
            new_instance.subject = subject
            new_instance.predicates = {}
            cls._instances[subject] = new_instance
        
        return cls._instances[subject]

    def __init__(self, subject: str, predicate: Union[str, RDFPredicate, List[RDFPredicate], List[List[any]]] = None, objects: Union[str, List[str]] = None):
        if predicate is None and objects is None:
            return
        
        elif (all(isinstance(p, list) for p in predicate)):
            for p in predicate:
                self._insert_statement(p[0], p[1])
        
        else:
            self._insert_statement(predicate, objects)

    def _insert_statement(self, predicate: Union[str, RDFPredicate, List[RDFPredicate], List[List[RDFPredicate]]]= None, objects: Union[str, List[str]] = None):
        if isinstance(predicate, str):
            is_objects_str = isinstance(objects, str)
            if not (is_objects_str or all(isinstance(o, str) for o in objects)):
                raise Exception(f"RDFStatement: {type(objects)} expected Union[str, List[str]]")
            if is_objects_str:
                objects = [objects]
            self._insert_on_property(predicate, objects)
        
        elif isinstance(predicate, RDFPredicate):
            self._insert_on_property(predicate.property, predicate.objects)

        elif isinstance(predicate, list):
            if not all(isinstance(p, RDFPredicate) for p in predicate):
                raise Exception(f"RDFStatement: {type(predicate)=} expected List[RDFPredicate]")
            for rdfp in predicate:
                self._insert_on_property(rdfp.property, rdfp.objects)
        elif predicate is None and objects is None:
            return
        else:
            raise Exception(f"RDFStatement:{predicate} {type(predicate)=} expected Union[str, RDFPredicate, List[RDFPredicate]]")
        
    def _insert_on_property(self, property: str, objects: list[str]):
        error = ["a", "rdf:type", "rdfs:type"]
        if property in error:
            if any(e in self.predicates.keys() for e in error):
                print(f"RDFStatement: Redefinition of a object type. {self} intended to overwrite {property} with {objects}")
                # raise Exception(f"RDFStatement: Redefinition of a object type.")
        if property in self.predicates:
            for obj in objects:
                if obj not in self.predicates[property]:
                    self.predicates[property].append(obj)
        else:
            self.predicates[property] = objects
    
    def to_ttl(self) -> str:
        ret = self.subject + " "
        for property, objects in self.predicates.items():
            ret += property + " " + ", ".join(objects) + ";\n\t"
        ret = ret[:-3]
        ret += "."
        return ret
    
    def __hash__(self):
        return id(self)
    
    def __eq__(self, other):
        if isinstance(other, RDFStatement):
            return self.__hash__() == other.__hash__()
        return False
    
class OrderSet:
    def __init__(self):
        self.data = {} 

    def insert(self, element):
        if element not in self.data:
            self.data[element] = len(self.data)

    def get_yield(self):
        # Devolver los elementos con su orden de inserción
        for element, order in self.data.items():
            yield (order, element)

ontology = OrderSet()
subjects = OrderSet()

nueva, notacion usando las siguientes data classes, esta para 
1 mantener la ontologia limpia es decir no tener duplicas de definicion de algo 
2 hacer un solo flush en el archivo ttl los datos, lo que implicara una mejora en el tiempo de la generacion del archivo


Ej 1:
```ttl
:11 a rdfs:Class;
	:famosoPor "numero chistoso"@es, "es primo"@es.
```

In [48]:
RDFStatement(":11", "a", "rdfs:Class")
RDFStatement(":11", ":famosoPor", ['"numero chistoso"@es', '"es primo"@es'])
print(RDFStatement(":11").to_ttl())

:11 a rdfs:Class;
	:famosoPor "numero chistoso"@es, "es primo"@es.


In [49]:
import unicodedata
def ttl_format(input_str: str) -> str:
    input_str = ''.join(c for c in unicodedata.normalize('NFKD', input_str) if not unicodedata.combining(c))
    input_str = input_str.lower().title().replace(" ", "_")
    return input_str

# Generar RDF Nationality

In [50]:
# TODO: HACER LA WEA
ontology.insert(RDFStatement("f1:Nationality", [["a", ["rdf:Class", "owl:Class"]],
                                            ["rdfs:label", "'A F1 nationality'@en"],
                                            ["rdfs:comment", "'A class representing a nationality related to F1'@en"]]))

unique_nationality = drivers['nationality'].unique()

i = 0 
for nationality in unique_nationality:
    #if i > 10: break
    subjects.insert(RDFStatement(f"f1:{ttl_format(nationality)}", [
        ["a", "f1:Nationality"],
        ["rdfs:label", f"'{nationality}'@en"]
    ]))
    i += 1

# Generar RDF Drivers

In [51]:
# Tenemos las nacionalidades de los pilotos, ahora crearemos el resto de datos

# Creamos la clase f1 Driver

ontology.insert(RDFStatement("f1:Driver", [
    ["a", ["rdf:Class", "owl:Class"]],
    ["rdfs:label", "'A F1 Driver'@en"],
    ["rdfs:comment", "'A class representing a F1 driver'@en"]
]))

ontology.insert(RDFStatement("f1:Code", [
    ["a", ["rdf:Class", "owl:Class"]],
    ["rdfs:label", "'A F1 Short 3 letter display'@en"],
    ["rdfs:comment", "'The usual way to represent a competitor in a race'@en"]
]))

# Añadimos cada driver con su nacionalidad correspondiente:
# Por cada driver, añadimos su nombre, su fecha de nacimiento, su nacionalidad y su número de piloto

#Muchos drivers number son simplemente /N, eliminar?
driver = []
i = 0
driver_ref = {}
for index, row in drivers.iterrows():
    #if i > 10 : break

    if ttl_format(row['driverRef']) == "Driver":
        continue
    
    wikidata_code = obtener_codigo_wikidata(row["url"])
    
    driver_ref[row['driverId']] = ttl_format(row['driverRef'])
    driver.append(RDFStatement(f"f1:{ttl_format(row['driverRef'])}", [
        ["a", "f1:Driver"],
        ["rdfs:label", f'"{row['forename']} {row['surname']}"@en'],
        ["f1:Nationality",f"f1:{ttl_format(row['nationality'])}"],
        ["f1:dateOfBirth", f"'{row['dob']}'^^xsd:date"],
    ]))

    if wikidata_code:
        RDFStatement(f"f1:{ttl_format(row['driverRef'])}", "owl:sameAs", f"wd:{wikidata_code}")

    if row['code'] != r'\N':
        RDFStatement(f"f1:{ttl_format(row['driverRef'])}", "f1:Code", f"'{row['code']}'")
        
    i += 1 

for d in driver:
    subjects.insert(d)


# Generar RDF Pistas

## Ontology needed

```ttl
f1:Circuit a rdf:Class, owl:Class;
    rdfs:label 'A F1 Circuit'@en;
    rdfs:comment 'A class representing a F1 circuit'@en.

f1:City a rdf:Class, owl:Class;
    rdfs:label 'A City'@en;
    rdfs:comment 'A City in the world related to f1'@en.

f1:hostCity a rdf:Property;
    rdfs:label 'City'@en;
    rdfs:comment 'a City hosting a F1 circuit'@en;
    rdfs:domain f1:Circuit.

f1:hostCountry a rdf:Property;
    rdfs:label 'Country'@en;
    rdfs:comment 'a Country hosting a F1 circuit'@en;
    rdfs:domain f1:Circuit.
```

In [52]:
ontology.insert(RDFStatement("f1:Circuit", [["a", ["rdf:Class", "owl:Class"]],
                                            ["rdfs:label", "'A F1 Circuit'@en"],
                                            ["rdfs:comment", "'A class representing a F1 circuit'@en"]]))

ontology.insert(RDFStatement("f1:Country", [["a", ["rdf:Class", "owl:Class"]],
                                            ["rdfs:label", "'A F1 Country'@en"],
                                            ["rdfs:comment", "'A class representing a F1 country'@en"]]))

ontology.insert(RDFStatement("f1:City",[["a", ["rdf:Class", "owl:Class"]],
                                        ["rdfs:label", "'A City'@en"],
                                        ["rdfs:comment", "'A City in the world related to f1'@en;"]]))

ontology.insert(RDFStatement("f1:hostCity",[["a", "rdf:Property"],
                                            ["rdfs:label", "'City'@en"],
                                            ["rdfs:comment", "'a City hosting a F1 circuit'@en"],
                                            ["rdfs:domain", "f1:Circuit"]]))

ontology.insert(RDFStatement("f1:hostCountry", [["a", "rdf:Property"],
                                                ["rdfs:label", "'Country'@en"],
                                                ["rdfs:comment", "'a Country hosting a F1 circuit'@en"],
                                                ["rdfs:domain", "f1:Circuit"]]))

* Example:
```ttl
f1:albert_park a f1:Circuit;
    rdfs:label 'Albert Park Grand Prix Circuit'@en;
    f1:hostCity f1:Melbourne;
    f1:hostCountry f1:Australia;
    geo:lat "-37.8497"^^xsd:float;
    geo:long "144.968"^^xsd:float;
```

In [53]:
cities = []
countries = []
sub1 = []
sub2 = []
circuits_ref = {}

i = 0
for index, row in circuits.iterrows():
    #if i > 10: break
    circuits_ref[row["circuitId"]] = f"{row["circuitRef"]}_Circuit"

    circuit = f"f1:{ttl_format(row["circuitRef"])}_Circuit"
    wkdta = obtener_codigo_wikidata(row["url"])

    sub2.append(
        RDFStatement(
            circuit,
            [
                ["rdf:type", "f1:Circuit"],
                ["rdfs:label", f"'{row["name"]}'@en"],
                ["f1:hostCity", f"f1:{ttl_format(row["location"])}"],
                ["f1:hostCountry", f"f1:{ttl_format(row["country"])}"],
                ["geo:lat", f'"{row["lat"]}"^^xsd:float'],
                ["geo:long", f'"{row["lng"]}"^^xsd:float']
            ]
        )
    )
    cities.append(ttl_format(row["location"]))
    countries.append(ttl_format(row["country"]))

    if wkdta:
        RDFStatement(circuit, "owl:sameAs", f"wd:{wkdta}")

cities = list(set(cities))
for city in cities:
    sub1.append(RDFStatement(f"f1:{city}", "a", "f1:City"))

countries = list(set(countries))
for country in countries:
    sub1.append(RDFStatement(f"f1:{country}", "a", "f1:Country"))

for i in sub1:
    subjects.insert(i)

for i in sub2:
    subjects.insert(i)


# Generar RDF Constructores

In [54]:
ontology.insert(RDFStatement("f1:Constructor", [
    ["a", ["rdf:Class", "owl:Class"]],
    ["rdfs:comment", "'A class representing an F1 constructor'@en"]
    ]))

i = 0
constructors_RDFS = []
constructors_ref = {}
for index,row in constructors.iterrows():
    
    wdLink = obtener_codigo_wikidata(row["url"])
    #if i > 10: break
    constructors_ref[row["constructorId"]] = f"{ttl_format(row['constructorRef'])}_Constructor"
    constructors_RDFS.append(RDFStatement(f"f1:{ttl_format(row['constructorRef'])}_Constructor", [
        ["a", "f1:Constructor"],
        ["rdfs:label", f"'{row['name']}'@en"],
        ["f1:Nationality", f"f1:{row["nationality"].replace(" ", "_")}"]
        ]))
    if wdLink:
        RDFStatement(f"f1:{ttl_format(row['constructorRef'])}_Constructor", "owl:sameAs", f"wd:{wdLink}")
    i+=1

for c in constructors_RDFS:
    subjects.insert(c)
        

# Seasons

In [55]:
seasons_data = []

ontology.insert(RDFStatement("f1:Season", [
    ["a", ["rdf:Class", "owl:Class"]],
    ["rdfs:comment", "'A class representing a F1 season'@en"]
    ]))

for index, row in seasons.iterrows():
    if row["year"] not in filter_years:
        continue
    
    wd_link = obtener_codigo_wikidata(row["url"])
    
    seasons_data.append(RDFStatement(f"f1:{row['year']}_Season", [
        ["a", "f1:Season"],
        ["rdfs:label", f"'{row['year']}'@en"]
    ]))
    
    if wd_link:
        RDFStatement(f"f1:{row['year']}_Season", "owl:sameAs", f"wd:{wd_link}")
    
for s in seasons_data:
    subjects.insert(s)

# Carreras

In [ ]:
races_id = {}
import asyncio

ontology.insert(RDFStatement(f"f1:Race",[
    ["a", ["rdf:Class", "owl:Class"]],
    ["rdfs:comment", "'A class representing a F1 race'@en"]
]))

i=0
races_data = []
races_ref = {}

ontology.insert(RDFStatement(f"f1:InCircuit", [
    ["a", ["rdf:Class"]],
    ["rdfs:comment", "'Done in the referenced circuit'@en"]
]))

ontology.insert(RDFStatement(f"f1:Round", [
    ["a", ["rdf:Class", "owl:Class"]],
    ["rdfs:comment", "'Number round of the race'@en"]
]))

ontology.insert(RDFStatement(f"f1:Round", [
    ["a", ["rdf:Class", "owl:Class"]],
    ["rdfs:comment", "'Number round of the race'@en"]
]))

for index, row in races.iterrows():
    # if row['year'] not in filter_years: continue
    #if i > 0: break
    races_ref[row['raceId']] = f"{row['year']}_{row['round']}"
    races_data.append(RDFStatement(f"f1:{row['year']}_{row['round']}", [
        ["a", "f1:Race"],
        ["f1:InCircuit", f"f1:{ttl_format(circuits_ref[row['circuitId']])}"],
        ["f1:date", f"'{row['date']}'^^xsd:date"],
        ["f1:Round", f"'{row['round']}'^^xsd:integer"],
        ["f1:InSeason", f"f1:{row['year']}_Season"]
    ]))
    if row['time'] != r'\N':
        races_data.append(RDFStatement(f"f1:{row['year']}_{row['round']}", "f1:time", f"'{row['time']}'^^xsd:time"))
    if row['url'] != r'\N':
        wd_link = obtener_codigo_wikidata(row['url'])
        if wd_link:
            RDFStatement(f"f1:{row['year']}_{row['round']}", "owl:sameAs", f"wd:{wd_link}")
    i += 1

for i in races_data:
    subjects.insert(i)

# Resultados

In [ ]:
### ESTO SE HIZO TAN SOLO PARA RESERVAR EL ESPACIO, SE REVISARA EL ARCHIVO result.csv
data_results = []
ontology.insert(RDFStatement("f1:Result", [
    ["a", ["rdf:Class", "owl:Class"]],
    ["rdfs:comment", "'A class representing a F1 result about a races'@en"]
]))

# ontology.insert()

status_dict = {}

for index, row in status.iterrows():
    status_dict[row['statusId']] = row['status']


for index, row in races_results.iterrows():
    if row['raceId'] not in races_ref.keys():
        continue
    
    if row['driverId'] not in driver_ref.keys():
        continue
    
    if row['constructorId'] not in constructors_ref.keys():
        continue
    
    data_results.append(RDFStatement(f"f1:{races_ref[row['raceId']]}_{driver_ref[row['driverId']]}", [
        ["a", "f1:Result"],
        ["f1:InRace", f"f1:{races_ref[row['raceId']]}"],
        ["f1:ForDriver", f"f1:{driver_ref[row['driverId']]}"],
        ["f1:Position", f"'{row['position']}'^^xsd:integer" if r"\N" != row["position"] else "f1:DNF"],
        ["f1:Points", f"'{row['points']}'^^xsd:float"],
        ["f1:ForCounstructor", f"f1:{constructors_ref[row['constructorId']]}"],
        ["f1:Time", f"'{row['time']}'" if r"\N" != row["time"] else "'No_Time'"],
        ["f1:Status", f"'{status_dict[row["statusId"]]}'" if r"\N" != row["statusId"] else "'No Status'"]
        ]))
    
    RDFStatement(f"f1:{races_ref[row['raceId']]}", "f1:Results", f"f1:{races_ref[row['raceId']]}_{driver_ref[row['driverId']]}")
    
for i in data_results:
    subjects.insert(i)


RDFStatement: Redefinition of a object type. RDFStatement(subject='f1:1978_14_Ertl', predicates={'a': ['f1:Result'], 'f1:InRace': ['f1:1978_14'], 'f1:ForDriver': ['f1:Ertl'], 'f1:Position': ['f1:DNF'], 'f1:Points': ["'0.0'^^xsd:float"], 'f1:ForCounstructor': ['f1:Ats_Constructor'], 'f1:Time': ["'No_Time'"], 'f1:Status': ["'Did not qualify'"]}) intended to overwrite a with ['f1:Result']
RDFStatement: Redefinition of a object type. RDFStatement(subject='f1:1962_5_Greene', predicates={'a': ['f1:Result'], 'f1:InRace': ['f1:1962_5'], 'f1:ForDriver': ['f1:Greene'], 'f1:Position': ['f1:DNF'], 'f1:Points': ["'0.0'^^xsd:float"], 'f1:ForCounstructor': ['f1:Lotus-Climax_Constructor'], 'f1:Time': ["'No_Time'"], 'f1:Status': ["'Withdrew'"]}) intended to overwrite a with ['f1:Result']
RDFStatement: Redefinition of a object type. RDFStatement(subject='f1:1961_5_Moss', predicates={'a': ['f1:Result'], 'f1:InRace': ['f1:1961_5'], 'f1:ForDriver': ['f1:Moss'], 'f1:Position': ['f1:DNF'], 'f1:Points': ["'0.

# Escritura de ontologia y subjects

In [58]:
with open(NAME_FILE, 'w') as f:
    f.write("""@base <https://seigenta.github.io/WATOS.github.io/data_test.ttl>.
@prefix f1: <http://ex.org/f1#>.
@prefix geo:  <http://www.opengis.net/ont/geosparql#>.
@prefix wd: <http://www.wikidata.org/entity/>.
@prefix owl: <http://www.w3.org/2002/07/owl#>.
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>. 
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.\n\n""")

    l = list(ontology.get_yield())
    l.sort()
    for e in l:
        f.write(e[1].to_ttl() + "\n\n")

    l = list(subjects.get_yield())
    l.sort()
    for e in l:
        f.write(e[1].to_ttl() + "\n\n")